In [2]:
import os
import time
import tensorflow as tf
import scipy.io as sio 
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.applications import VGG19
# from keras.applications import MobileNetv2
import keras
from keras.preprocessing.image import ImageDataGenerator

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\api\preprocessing\image\__init__.py)

In [1]:
# load data
train_dir = "garbage_classification"

batch_size = 64
img_height = 32
img_width = 32

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,  # 使用20%的数据进行验证
    subset="training",
    seed=123,  # 固定随机种子，确保每次生成的数据集相同
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True
)

NameError: name 'tf' is not defined

In [5]:
train_dir = "dataset_aug_6"
val_dir = "dataset_aug_6"

batch_size = 64
img_height = 224
img_width = 224

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training',
    shuffle = True
)

val_dataset = train_datagen.flow_from_directory(
    val_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'validation',
    shuffle = True
)

Found 4201 images belonging to 6 classes.
Found 1047 images belonging to 6 classes.


In [7]:
# define model

from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

# early_stopping : val_loss 連續不降低時，結束訓練
early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1,restore_best_weights=True,mode='auto')
# reduce_lr : val_loss 連續不降低時，減少learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='auto',
                            epsilon=0.0001, cooldown=0, min_lr=0)

checkpoint = ModelCheckpoint('MobileNetv2_aug6_v2.h5', monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

# pre_model = MobileNet(input_shape = (128, 128, 3), include_top=False, pooling='avg')
pre_model = MobileNet(input_shape = (img_height, img_width, 3), weights='imagenet')
pre_model.trainable = True

transfer_model = keras.models.Sequential()
transfer_model.add(pre_model)
transfer_model.add(keras.layers.Flatten())
transfer_model.add(keras.layers.Dense(1024, activation='relu'))
transfer_model.add(keras.layers.Dropout(0.5))
transfer_model.add(keras.layers.Dense(512, activation='relu'))
transfer_model.add(keras.layers.Dense(len(train_dataset.class_indices), activation='softmax'))

# pre_model.trainable = True

transfer_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functi  (None, 1000)              4253864   
 onal)                                                           
                                                                 
 flatten_2 (Flatten)         (None, 1000)              0         
                                                                 
 dense_6 (Dense)             (None, 1024)              1025024   
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_7 (Dense)             (None, 512)               524800    
                                                                 
 dense_8 (Dense)             (None, 6)                 3078      
                                                      

In [8]:
# optimizer = keras.optimizers.Adam(lr=0.0001)
transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
history = transfer_model.fit(train_dataset, validation_data = val_dataset, epochs=50, callbacks=[early_stopping,checkpoint,reduce_lr])

Epoch 1/50
66/66 [==============================] - ETA: 0s - loss: 0.8491 - accuracy: 0.8010
Epoch 1: val_loss improved from inf to 2.74614, saving model to MobileNetv2_aug6_v2.h5
66/66 [==============================] - 281s 4s/step - loss: 0.8491 - accuracy: 0.8010 - val_loss: 2.7461 - val_accuracy: 0.2235 - lr: 0.0010
Epoch 2/50
66/66 [==============================] - ETA: 0s - loss: 0.2472 - accuracy: 0.9467
Epoch 2: val_loss improved from 2.74614 to 2.06709, saving model to MobileNetv2_aug6_v2.h5
66/66 [==============================] - 265s 4s/step - loss: 0.2472 - accuracy: 0.9467 - val_loss: 2.0671 - val_accuracy: 0.5960 - lr: 0.0010
Epoch 3/50
66/66 [==============================] - ETA: 0s - loss: 0.1819 - accuracy: 0.9572
Epoch 3: val_loss improved from 2.06709 to 0.77767, saving model to MobileNetv2_aug6_v2.h5
66/66 [==============================] - 246s 4s/step - loss: 0.1819 - accuracy: 0.9572 - val_loss: 0.7777 - val_accuracy: 0.7708 - lr: 0.0010
Epoch 4/50
64/66 [==

KeyboardInterrupt: 